This document is built on top of Pablo's toy example.

# Hamiltonian
The Gross-Pitaevskii Equation (GPE) is a manifestation of the Hamiltonian principle in quantum mechanics, specifically tailored for Bose-Einstein Condensates (BECs).

The GPE is a specific form of a nonlinear Schrödinger equation that takes into account:
- the kinetic energy of the particles
- the potential energy from an external trapping potential
- the interactions among the particles in the BEC

In quantum mechanics, the Hamiltonian represents the total energy of a system, encompassing both kinetic and potential energies. It is central to the formulation of the Schrödinger equation, which describes how the quantum state of a physical system changes over time. The Hamiltonian operator applied to a wavefunction yields the energy of the system.

The GPE, through its Hamiltonian, captures the essential physics of BECs by considering the contributions of kinetic energy, external potential, and particle interactions. It predicts the spatial and temporal evolution of the condensate's wavefunction, which, in turn, describes the density and phase of the condensate across space and time.

Consider the following toy model to solve:

$$\displaystyle \left(-\frac{d^2}{dx^2}+k x^2+q \rho(x)^\alpha\right)\phi_i(x) = \lambda_i \phi_i(x),$$

where:
- $\frac{d^2}{dx^2}$ represents the the kinetic energy of particles computed via Laplacian operator indicating the second spatial derivatives, and is implemented via ``generate_second_derivative_matrix``.
- $k x^2$ represents the external potential energy due to external fields or implemented traps.
- $\rho(x)^\alpha$ is called self interaction potential and is implemented via ``self_interaction_potential``.
- $\displaystyle \rho(x) = \sum_{i}^{W}|\phi_i(r)|^2$, with $W$ representing the number of wavefuntions that satisfy the normalization conditions. This behavior is implemented via ``rho_maker``.




In [ ]:
import numpy as np
import pandas as pd
import random
import scipy.optimize as op
import matplotlib.pyplot as plt
from scipy.stats import qmc
import time
import timeit
from tqdm import tqdm

import matplotlib
import warnings
warnings.filterwarnings("ignore")
''' Matplotlib settings '''
plt.rcParams['figure.figsize'] = [15, 5]
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 18}
matplotlib.rc('font', **font)

## Utils
Groups all the required functions in the same cell.

In [ ]:
def generate_second_derivative_matrix(xgrid):
    N = len(xgrid)
    dx = xgrid[1]-xgrid[0]
    # Generate the matrix for the second derivative using a five-point stencil
    main_diag = np.ones(N) * (-5.0 / 2 / dx**2)
    off_diag = np.ones(N - 1) * 4 / 3 / dx**2
    off_diag2 = np.ones(N - 2) * (-1.0 / (12 * dx**2))
    D2 = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1) + \
         np.diag(off_diag2, k=2) + np.diag(off_diag2, k=-2)
    return D2

def harmonic_potential_matrix(xgrid):
    return np.diag(xgrid**2)

def self_interaction_potential(rho, alpha):
    return np.diag(rho**alpha)

def list_normalizing(wf_list, dx):
    return [wf/np.linalg.norm(wf)*np.sign(wf_list[0][int(len(wf_list[0])/2)])*1/np.sqrt(dx) for wf in wf_list]

def H_solver(kappa, q, alpha, D2Mat, harmonic_matrix, rho, dx, total_wfs):
    # kinetic + external + interaction
    H = -D2Mat + kappa*harmonic_matrix + q*self_interaction_potential(rho, alpha)
    # Eigenvectors and eigenvalues
    evals, evects = np.linalg.eigh(H)
    return [evals[0:total_wfs], list_normalizing(evects.T[0:total_wfs], dx)]

def rho_maker(wf_list):
    sq_list=[wf**2 for wf in wf_list]
    return np.sum(sq_list, axis=0)

# Generate the dataset



Generate a grid of parameters in the following range:
- $κ$: [0.5, 3]
- $q$: [-2, 2]
- $a$: [0.5, 3]

In [ ]:
ks_step = 0.2
ks = np.arange(start=0.5, stop=3+ks_step, step=ks_step)
qs_step = 0.2
qs = np.arange(start=-2, stop=2+qs_step, step=qs_step)
as_step = ks_step
alphas = np.arange(start=0.5, stop=3+as_step, step=as_step)
print(ks.shape, qs.shape, alphas.shape)

(14,) (21,) (14,)


Generate a set of Eigen Vectors, Eigen Values and Densities from the initial starting points at $k=1.25,\ q=1,\ \alpha=1.25$ (the middle values in the range).

Consider a reduced N (50 instead of 300).

In [ ]:
x_max = 15.0
N = 100
x = np.linspace(-x_max, x_max, N)
total_wfs = 5
dx = x[1] - x[0]
# Get static inputs
D20 = generate_second_derivative_matrix(x)
harmonic_matrix = harmonic_potential_matrix(x)
# Get initial guess
kappa, q, alpha = 1.25, 1, 1.25
Solution_init = H_solver(kappa, q, alpha, D20, harmonic_matrix, np.zeros(len(x)), dx, total_wfs)
# Get initial density estimate
rho_init = rho_maker(Solution_init[1])
#rho_init = 1/2*(rho_init+np.flip(rho_current))
eigen_vals_init = Solution_init[0]

Generate the target densities for each $k_i,\ q_i,\ \alpha_i$ from the grid.

In [ ]:
# Stopping criteria
max_iteration = 500
tolerance = 10**(-5)
# Loop
mixing_rate = 0.15
df_g = {}
for ki in ks:
  df_g[ki] = {}
  for qi in qs:
    df_g[ki][qi] = {}
    for ai in alphas:
      # Copy initial estimate
      rho_current_i = np.copy(rho_init)
      eigen_vals_current_i = np.copy(rho_init)
      # Repeat Hartree–Fock (iterative) method until no change is observed in the density
      for k in range(max_iteration):
          eigen_vals_old = np.copy(eigen_vals_current_i)
          # Solves equations for the new rho_current_i with user-defined q and alpha
          Solution_current_i = H_solver(ki, qi, ai, D20, harmonic_matrix, rho_current_i, dx, total_wfs)
          # Extracts eigenvalues and verifies the stopping criteria
          eigen_vals_current = Solution_current_i[0]
          rho_current_i = rho_maker(Solution_current_i[1])
          # Stop if no significant change was observed
          if (max(np.abs(eigen_vals_current_i - eigen_vals_old)) < tolerance):
              break
          else:
            # Updates rho_current_i as a linear combination of the new and the previous densities
            rho_current_i = rho_current_i*mixing_rate+rho_current_i*(1-mixing_rate)
            rho_current_i = 1/2*(rho_current_i+np.flip(rho_current_i))

      dict_kqa = {"x": x} | {"Evec{}".format(evi): evec for evi, evec in enumerate(Solution_init[1])}
      dict_kqa = dict_kqa | {"Eval{}".format(evi): eval for evi, eval in enumerate(eigen_vals_init)}
      dict_kqa = dict_kqa | {"D_init": np.copy(rho_init), "D_target": rho_current_i}
      df_g[ki][qi][ai] = pd.DataFrame.from_dict(dict_kqa)

    df_g[ki][qi] = pd.concat(df_g[ki][qi])
  df_g[ki] = pd.concat(df_g[ki])
df_g = pd.concat(df_g)

Reset index.

In [ ]:
df_g = df_g.reset_index().rename(columns={"level_0": "k",	"level_1": "q",	"level_2": "a",	"level_3": "i"})
display(df_g.head(3))
display(df_g.tail(3))

,k,q,a,i,x,Evec0,Evec1,Evec2,Evec3,Evec4,Eval0,Eval1,Eval2,Eval3,Eval4,D_init,D_target
0,0.5,-2.0,0.5,0,-15.000000,3.195694e-32,3.680740e-31,-6.080023e-30,-4.462948e-29,2.196918e-28,1.117796,3.352447,5.584314,7.811627,10.032689,5.029340e-56,2.191024e-42
1,0.5,-2.0,0.5,1,-14.696970,5.710792e-30,6.426360e-30,-2.975962e-29,-2.674603e-28,1.380744e-27,1.117796,3.352447,5.584314,7.811627,10.032689,1.978949e-54,6.281205e-41
2,0.5,-2.0,0.5,2,-14.393939,8.906518e-29,7.625833e-29,-3.738771e-29,-1.058690e-27,6.238049e-27,1.117796,3.352447,5.584314,7.811627,10.032689,4.004923e-53,1.233579e-39


,k,q,a,i,x,Evec0,Evec1,Evec2,Evec3,Evec4,Eval0,Eval1,Eval2,Eval3,Eval4,D_init,D_target
411597,3.1,2.0,3.1,97,14.393939,-3.804376e-20,-2.404824e-20,-5.577405e-20,-8.131785e-20,-1.019044e-19,1.117796,3.352447,5.584314,7.811627,10.032689,2.213348e-38,9.115602e-46
411598,3.1,2.0,3.1,98,14.696970,2.048614e-20,1.224887e-20,3.095544e-20,4.483504e-20,5.375446e-20,1.117796,3.352447,5.584314,7.811627,10.032689,6.427679e-39,2.491268e-46
411599,3.1,2.0,3.1,99,15.000000,-1.431229e-20,-9.210530e-21,-2.205266e-20,-2.868882e-20,-3.652338e-20,1.117796,3.352447,5.584314,7.811627,10.032689,2.933001e-39,1.428535e-47


Stores the dataset on my drive.

In [ ]:
# Mount Google Drive in Colab
from google.colab import drive
drive.mount('/content/drive')
# Write the Dataset to CSV
file_path = '/content/drive/MyDrive/MSU/FRIB - W. Nazarewicz/x_max-15_N-100_wft-5_ki-125_qi-1_ai-123_by-01.csv'
df_g.to_csv(file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
